## ODM based on date interval - 2025

In [4]:
import pandas as pd
import os

ETM_folder = "Latest_ETM_Data"
ODM_folder = "ODM_Files"

In [7]:
all_zonal_centroids = range(1, 3040)

all_pairs = pd.MultiIndex.from_product(
    [all_zonal_centroids, all_zonal_centroids],
    names=['source_zonal_centroid_number', 'destination_zonal_centroid_number']
)

combined_od_matrix = pd.DataFrame(index=all_zonal_centroids, columns=all_zonal_centroids).fillna(0)

start_date = int(input("Enter the start date: "))
end_date = int(input("Enter the end date: "))

for date in range(start_date, end_date + 1):
    date_str = f"{date:02}"
    try:
        df = pd.read_csv(f'./{ETM_folder}/Final_Ticket_data_2024-07-{date_str}.csv')

        # Group the data and sum the passenger counts
        grouped = df.groupby(['source_zonal_centroid_number', 'destination_zonal_centroid_number'])['passenger_count'].sum()

        od_matrix = grouped.reindex(all_pairs, fill_value=0).unstack(fill_value=0)
        
        combined_od_matrix += od_matrix

        print(f"Processed data for 2024-07-{date_str}")

    except FileNotFoundError:
        print(f"File not found for 2024-07-{date_str}, skipping.")
    except Exception as e:
        print(f"An error occurred for 2024-07-{date_str}: {e}")


combined_od_matrix = combined_od_matrix.fillna(0)
boarding = combined_od_matrix.sum(axis=1)
combined_od_matrix = pd.concat([combined_od_matrix, boarding.rename('Boarding')], axis=1)

Enter the start date: 03
Enter the end date: 04
Processed data for 2024-07-03
Processed data for 2024-07-04
Combined OD matrix saved successfully.


In [9]:
combined_od_matrix.loc['Alighting'] =combined_od_matrix.sum(axis=0)

In [10]:
combined_od_matrix

,1,2,3,4,5,6,7,8,9,10,...,3031,3032,3033,3034,3035,3036,3037,3038,3039,Boarding
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,65679
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,33,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3036,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3037,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3038,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3039,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
# Save the combined OD matrix
combined_od_matrix.to_csv(f'{ODM_folder}/Combined_ODM_2024-07.csv', index=True)